In [5]:
import os, re
import pandas as pd
from IPython.display import display

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
file_path = r"D:\Python\PyTorch\WajahBuku\data\books\books.csv"
df = pd.read_csv(file_path)

display(df.head())
print("\n")
df.info()
print("\n")
display(df.describe())
print("\n")
display(df.describe(include='object'))

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6810 entries, 0 to 6809
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   isbn13          6810 non-null   int64  
 1   isbn10          6810 non-null   object 
 2   title           6810 non-null   object 
 3   subtitle        2381 non-null   object 
 4   authors         6738 non-null   object 
 5   categories      6711 non-null   object 
 6   thumbnail       6481 non-null   object 
 7   description     6548 non-null   object 
 8   published_year  6804 non-null   float64
 9   average_rating  6767 non-null   float64
 10  num_pages       6767 non-null   float64
 11  ratings_count   6767 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 638.6+ KB




,isbn13,published_year,average_rating,num_pages,ratings_count
count,6.810000e+03,6804.000000,6767.000000,6767.000000,6.767000e+03
mean,9.780677e+12,1998.630364,3.933284,348.181026,2.106910e+04
std,6.068911e+08,10.484257,0.331352,242.376783,1.376207e+05
min,9.780002e+12,1853.000000,0.000000,0.000000,0.000000e+00
25%,9.780330e+12,1996.000000,3.770000,208.000000,1.590000e+02
50%,9.780553e+12,2002.000000,3.960000,304.000000,1.018000e+03
75%,9.780810e+12,2005.000000,4.130000,420.000000,5.992500e+03
max,9.789042e+12,2019.000000,5.000000,3342.000000,5.629932e+06


,isbn10,title,subtitle,authors,categories,thumbnail,description
count,6810,6810,2381,6738,6711,6481,6548
unique,6810,6398,2009,3780,567,6481,6474
top,9042003405,The Lord of the Rings,A Novel,Agatha Christie,Fiction,http://books.google.com/books/content?id=2lVyR...,No Marketing Blurb
freq,1,11,226,37,2588,1,6


In [6]:
#  Definisi normalisasi dan mapping rules (sesuaikan rules jika perlu)
def normalize_text(s):
    if pd.isna(s):
        return ""
    s = str(s).strip()
    s = re.sub(r'\s+', ' ', s)
    s = s.replace('&', ' and ')
    s = s.replace('/', ' ')
    s = re.sub(r"[^\w\s\-\.,()']", '', s)
    return s.lower()

mapping_rules = [
    (["juvenile", "children", "young adult", "babytime", "babysitters club"], "Children & Young Adult"),
    (["biograph", "autobiograph", "biography"], "Biography & Autobiography"),
    (["history", "world war", "tobruk", "europe", "france", "india", "china", "japan", "canada", "england", "boston", "new england"], "History & Geography"),
    (["literary criticism", "literary collections", "literary", "literature", "authors", "poetry", "short stories", "essays", "poets"], "Literature & Criticism"),
    (["philosophy", "existentialism", "empiricism", "confucianism"], "Philosophy"),
    (["religion", "bible", "christian", "buddhism", "demonology", "cults", "psychoanalysis and religion"], "Religion & Spirituality"),
    (["comic", "comics", "graphic", "graphic novels", "comic books"], "Comics & Graphic Novels"),
    (["drama", "play", "plays", "motion picture plays", "theatre"], "Drama & Performing Arts"),
    (["science fiction", "sci fi", "sciencefiction", "star trek", "dystopia", "fantasy", "fantasy fiction", "dragons", "elves"], "Fantasy & Science Fiction"),
    (["science", "physics", "chemistry", "biology", "astron", "cosmology", "mathematics"], "Science & Mathematics"),
    (["business", "econom", "finance", "capitalism", "banks", "banking", "business enterprises"], "Business & Economics"),
    (["social", "political", "political science", "politics", "apartheid", "antislavery", "social action", "social science"], "Social Sciences"),
    (["psychology", "mental", "behavior", "bereavement", "autonomy (psychology)"], "Psychology & Self-Help"),
    (["self-help", "self help", "conduct of life"], "Self-Help & Personal Development"),
    (["health", "medical", "fitness", "cerebrovascular", "alzheimer", "obesity", "cancer"], "Health & Medicine"),
    (["cook", "cooking", "cookery", "cookbooks"], "Cooking & Food"),
    (["art", "photography", "architecture", "design", "artists", "museum"], "Art & Design"),
    (["music", "rock musicians", "cellists"], "Music"),
    (["sports", "recreation", "games", "sports & recreation"], "Sports & Recreation"),
    (["comput", "technology", "engineering", "programming", "computer", "technology & engineering"], "Computers & Technology"),
    (["law"], "Law & Government"),
    (["reference", "dictionary", "guide"], "Reference"),
    (["travel", "paris", "japan", "china", "india", "europe"], "Travel & Geography"),
    (["education", "study aids", "college", "high schools", "teacher"], "Education & Study Aids"),
    (["craft", "crafts", "hobbies", "gardening"], "Hobbies & Crafts"),
    (["humor", "wit", "funny", "humorous"], "Humor"),
    (["true crime", "crime", "murder", "detective", "intelligence service", "mystery", "thriller"], "Crime, Mystery & Thriller"),
    (["pet", "animal", "animals", "sanctuary", "birds", "cats", "dogs"], "Animals & Nature"),
]

def map_category_to_toplevel(cat):
    s = normalize_text(cat)
    for keywords, tgt in mapping_rules:
        for kw in keywords:
            if kw in s:
                return tgt
    # heuristik fallback
    if "fiction" in s:
        return "Fiction & Literature"
    if "biograph" in s:
        return "Biography & Autobiography"
    if s == "" or s.strip() == "":
        return "Unknown / Missing"
    return "Other / Misc"

# Terapkan mapping IN-PLACE pada kolom 'categories' (supaya jumlah kolom tetap sama)
if 'categories' not in df.columns:
    raise KeyError("Kolom 'categories' tidak ditemukan di DataFrame.")

df['categories'] = df['categories'].apply(map_category_to_toplevel)

#  Simpan file baru
out_drive_dir = r'D:\Python\PyTorch\WajahBuku\data\books'
os.makedirs(out_drive_dir, exist_ok=True)
out_drive = os.path.join(out_drive_dir, 'books_processed.csv')
df.to_csv(out_drive, index=False)

# Tampilkan ringkasan singkat hasil mapping
print("\nTop category counts:")
print(df['categories'].value_counts().to_string())


Top category counts:
categories
Fiction & Literature                2649
Children & Young Adult               711
Other / Misc                         565
Biography & Autobiography            406
Literature & Criticism               377
History & Geography                  307
Science & Mathematics                183
Comics & Graphic Novels              168
Philosophy                           166
Art & Design                         166
Religion & Spirituality              158
Drama & Performing Arts              140
Unknown / Missing                     99
Business & Economics                  80
Health & Medicine                     62
Psychology & Self-Help                61
Fantasy & Science Fiction             60
Cooking & Food                        51
Computers & Technology                49
Sports & Recreation                   47
Humor                                 45
Travel & Geography                    44
Crime, Mystery & Thriller             43
Self-Help & Personal Dev